In [1]:
import os
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings

In [12]:
CHROMA_PATH = "../chroma_minilm"
DATA_PATH = "../documents"
model = 'tazarov/all-minilm-l6-v2-f32:latest'

In [13]:
from __future__ import annotations

import re
from typing import Any, List, Optional

from langchain_text_splitters.base import Language, TextSplitter

def _split_text_with_regex(
    text: str, separator: str, keep_separator: bool
) -> List[str]:
    # Now that we have the separator, split the text
    if separator:
        if keep_separator:
            # The parentheses in the pattern keep the delimiters in the result.
            _splits = re.split(f"({separator})", text)
            splits = [_splits[i] + _splits[i + 1] for i in range(1, len(_splits), 2)]
            if len(_splits) % 2 == 0:
                splits += _splits[-1:]
            splits = [_splits[0]] + splits
        else:
            splits = re.split(separator, text)
    else:
        splits = list(text)
    return [s for s in splits if s != ""]

class RecursiveRegexTextSplitter(TextSplitter):
    """Splitting text by recursively look at characters.

    Recursively tries to split by different characters to find one
    that works.
    """

    def __init__(
        self,
        separators: Optional[List[str]] = None,
        keep_separator: bool = False,
        is_separator_regex: bool = False,
        match_str: Optional[str] = None,
        **kwargs: Any,
    ) -> None:
        """Create a new TextSplitter."""
        super().__init__(keep_separator=keep_separator, **kwargs)
        self._separators = separators or ["\n\n", "\n", " ", ""]
        self._is_separator_regex = is_separator_regex
        self.match_str = match_str

    def _split_text(self, text: str, separators: List[str]) -> List[str]:
        """Split incoming text and return chunks."""
        final_chunks = []
        # Get appropriate separator to use
        separator = "\n"
        new_separators = []
        for i, _s in enumerate(separators):
            _separator = _s if self._is_separator_regex else re.escape(_s)
            if _s == "":
                separator = _s
                break
            if re.search(_separator, text):
                separator = _s
                new_separators = separators[i + 1 :]
                break

        _separator = separator if self._is_separator_regex else re.escape(separator)
        splits = _split_text_with_regex(text, _separator, self._keep_separator)

        # Now go merging things, recursively splitting longer texts.
        _good_splits = []
        _separator = "" if self._keep_separator else separator

        for s in splits:
            if bool(re.match(self.match_str, s)) == False:
                _good_splits.append(s)
            else:
                if _good_splits:
                    merged_text = self._merge_splits(_good_splits, _separator)
                    final_chunks.extend(merged_text)
                    print(merged_text)
                    _good_splits = [s]
                if not new_separators:
                    final_chunks.append(s)
                else:
                    other_info = self._split_text(s, new_separators)
                    final_chunks.extend(other_info)
        if _good_splits:
            merged_text = self._merge_splits(_good_splits, _separator)
            final_chunks.extend(merged_text)
        return final_chunks

    def split_text(self, text: str) -> List[str]:
        return self._split_text(text, self._separators)

In [14]:
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

# play around with various chunk sizes and overlapping
def split_documents(documents: list[Document]):
    text_splitter = RecursiveRegexTextSplitter(
        chunk_size=1600,
        chunk_overlap=160,
        length_function=len,
        is_separator_regex=False,
        match_str=r'\b[0-9]{1,}\b\.'
    )
    return text_splitter.split_documents(documents)


def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function = OllamaEmbeddings(model=model)
    )

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")


def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks


def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

In [15]:
documents = load_documents()
chunks = split_documents(documents)

print(chunks[:5])

Created a chunk of size 1613, which is longer than the specified 1600
Created a chunk of size 1663, which is longer than the specified 1600


['most common association with 14–36% of individuals\nwith OA/TOF having two or more VACTERL anoma-lies [Torfs et al., 1995; Rittler et al., 1996]. Other as-sociations described include CHARGE, PHAVER, andSCHISIS associations and Goldenhar syndrome [Chit-tmittrapap et al., 1989; Powell et al., 1993; Sutphen etal., 1995; Tellier et al., 1998]. OA often occurs in thepresence of a number of chromosomal anomalies, inparticular trisomy 18 and 21 [Benady and Harris,1969; Harris et al., 1995; Torfs et al., 1995; Beasley etal., 1997]. Children with OA frequently have additionalmedical problems following successful surgical repairof the primary malformation. Recurrent respiratory in-fections [Dudley and Phelan, 1976; Checuti and Phe-lan, 1993b], swallowing difficulties [Smith and Beck,1985; Checuti and Phelan, 1993a] and intractable gas-tro-oesophageal reflux [Ashcraft et al., 1977; Parker etal., 1979; Jolley et al., 1980] are common causes of mor-bidity.\nOA/ TOF may represent part of a spectr

In [16]:
add_to_chroma(chunks)

Number of existing documents in DB: 0
👉 Adding new documents: 847
